https://towardsdatascience.com/lets-code-a-neural-network-in-plain-numpy-ae7e74410795

In [109]:
import numpy as np
from pprint import pprint

In [112]:
class Sigmoid:
    def __call__(self, Z):
        return 1 / (1 + np.exp(-Z))
    
    def grad(self, dA, Z):
        sig = self(Z)
        return dA * sig * (1 - sig)
    
class Relu:
    def __call__(self, Z):
        return np.maximum(0, Z)
    
    def grad(self, dA, Z):
        dZ = np.array(dA, copy=True)
        dZ[Z <= 0] = 0
        return dZ
    
ACT = {'relu': Relu(), 'sigmoid': Sigmoid()}

class Layer:
    def __init__(self, in_size, out_size, act='relu'):
        self.w = np.random.normal(scale=0.1, size=(out_size, in_size))
        self.b = np.random.normal(scale=0.1, size=(out_size, 1))
        self.act = ACT[act]
        
    def __repr__(self):
        return f'Layer{*self.w.T.shape, self.act}'
    
    def forward(self, A):
        Z = self.w @ A + self.b
        return self.act(Z), Z
         
class Model:
    def __init__(self, sizes, seed=42):
        np.random.seed(seed)
        self.layers = []
        for i in range(len(sizes) - 1):
            in_size, out_size = sizes[i: i + 2]
            self.layers.append(Layer(in_size, out_size))
        self.layers.append(Layer(sizes[-1], 1, 'sigmoid'))
        
    def __repr__(self):
        return '\n'.join([str(l) for l in self.layers])
    
    def forward(self, X):
        memory = {}
        for i, l in enumerate(self.layers):
            X, _ = l.forward(X)
        return X
            
        
        
m = Model([2])
m

Layer(2, 1, <__main__.Sigmoid object at 0x7f7ee80fd518>)

In [95]:
X = np.array([[1], [2]])
m.forward(X)

array([[0.51058178]])

In [111]:
def init_layers(nn_architecture, seed = 42):
    np.random.seed(seed)
    number_of_layers = len(nn_architecture)
    params_values = {}

    for idx, layer in enumerate(nn_architecture):
        layer_idx = idx + 1
        layer_input_size = layer["input_dim"]
        layer_output_size = layer["output_dim"]
        
        params_values['W' + str(layer_idx)] = np.random.randn(
            layer_output_size, layer_input_size) * 0.1
        params_values['b' + str(layer_idx)] = np.random.randn(
            layer_output_size, 1) * 0.1
        
    return params_values

def sigmoid(Z):
    return 1/(1+np.exp(-Z))

def relu(Z):
    return np.maximum(0,Z)

def sigmoid_backward(dA, Z):
    sig = sigmoid(Z)
    return dA * sig * (1 - sig)

def relu_backward(dA, Z):
    dZ = np.array(dA, copy = True)
    dZ[Z <= 0] = 0;
    return dZ;

def single_layer_forward_propagation(A_prev, W_curr, b_curr, activation="relu"):
    Z_curr = np.dot(W_curr, A_prev) + b_curr
    
    if activation is "relu":
        activation_func = relu
    elif activation is "sigmoid":
        activation_func = sigmoid
    else:
        raise Exception('Non-supported activation function')
        
    return activation_func(Z_curr), Z_curr

def full_forward_propagation(X, params_values, nn_architecture):
    memory = {}
    A_curr = X
    
    for idx, layer in enumerate(nn_architecture):
        layer_idx = idx + 1
        A_prev = A_curr
        
        activ_function_curr = layer["activation"]
        W_curr = params_values["W" + str(layer_idx)]
        b_curr = params_values["b" + str(layer_idx)]
        A_curr, Z_curr = single_layer_forward_propagation(A_prev, W_curr, b_curr, activ_function_curr)
        
        memory["A" + str(idx)] = A_prev
        memory["Z" + str(layer_idx)] = Z_curr
       
    return A_curr, memory

nn_architecture = [
    {"input_dim": 2, "output_dim": 2, "activation": "relu"},
    {"input_dim": 2, "output_dim": 1, "activation": "sigmoid"},
]
param_values = init_layers(nn_architecture)
pprint(param_values)

X = np.array([[1], [2]])
pprint(full_forward_propagation(X, param_values, nn_architecture))



{'W1': array([[ 0.04967142, -0.01382643],
       [ 0.06476885,  0.15230299]]),
 'W2': array([[0.15792128, 0.07674347]]),
 'b1': array([[-0.02341534],
       [-0.0234137 ]]),
 'b2': array([[-0.04694744]])}
(array([[0.49490088]]),
 {'A0': array([[1],
       [2]]),
  'A1': array([[0.        ],
       [0.34596113]]),
  'Z1': array([[-0.00139678],
       [ 0.34596113]]),
  'Z2': array([[-0.02039718]])})


In [118]:
m = Model([2, 2])

In [122]:
m.forward(X)

array([[0.49490088]])